## Sequential Data 준비

In [ ]:
import torch
import torch.nn as nn

# Hyperparameters
input_size = 5  # 입력 벡터의 크기 --> 단어(word) 5차원
hidden_size = 10  # 히든 상태 크기 --> 문맥의 정보 10차원
sequence_length = 6  # 시퀀스 길이 --> 단어의 수
batch_size = 3  # 배치 크기

In [ ]:
# 임의의 입력 데이터 (배치 크기, 시퀀스 길이, 입력 차원) x data
inputs = torch.randn(sequence_length, batch_size, input_size)
inputs.shape # 시퀀스 길이, 배치 크기, 입력(임베딩) 차원

torch.Size([6, 3, 5])

## RNN 모델 사용하기 기본

RNN 공식문서 : https://pytorch.org/docs/stable/generated/torch.nn.RNN.html

데이터의 차원과 모델의 입출력 차원을 주의깊게 확인합시다

In [ ]:
# RNN 모델 build
rnn = nn.RNN(input_size, hidden_size, batch_first=False)
# batch_first=False가 디폴트이므로 생략 가능

# RNN 실행
# output: 모든 타임스텝에 대한 RNN의 출력
# hidden: 마지막 타임스텝의 히든 상태
output, hidden = rnn(inputs)

print("Output shape:", output.shape)  # (seq_len, batch, hidden_size)
print("Hidden shape:", hidden.shape)  # (num_layers, batch, hidden_size)

Output shape: torch.Size([6, 3, 10])
Hidden shape: torch.Size([1, 3, 10])


In [ ]:
# RNN 모델
rnn = nn.RNN(input_size, hidden_size, batch_first=True)

# RNN 실행
# output: 모든 타임스텝에 대한 RNN의 출력
# hidden: 마지막 타임스텝의 히든 상태
output, hidden = rnn(inputs.transpose(0, 1)) # 6, 3, 5 -> 3, 6, 5 (b, s, d)

print("Output shape:", output.shape)  # (batch_size, sequence_length, hidden_size)
print("Hidden shape:", hidden.shape)  # (num_layers, batch_size, hidden_size)

Output shape: torch.Size([3, 6, 10])
Hidden shape: torch.Size([1, 3, 10])


### RNN 모델 학습해보기

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Hyperparameters
## model parameter
input_size = 5 # 한단어의 차원
hidden_size = 10 # 모든 문장의 의미를 담은 vector 10차원

## data parameter
sequence_length = 6
batch_size = 3
num_classes = 2 # 0, 1

## training parameter
learning_rate = 0.01
num_epochs = 20

# 간단한 RNN 분류 모델
# 클래스의 특징/장점
# 유사한, 기능을 가지는 코드들을 묶어서 관리
# 함수(메서드)와 변수(어트리뷰트)를 같이 묶을 수 있다.
# 상속을 받을 수 있다. --> 미리 만들어져 있는 코드

class SimpleRNN(nn.Module): #객체지향, 절차형, 함수형 (데이터분석, 웹서버)
    def __init__(self, input_size, hidden_size, num_classes):
        super(SimpleRNN, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        _, hidden = self.rnn(x)
        out = self.fc(hidden[0])  # 마지막 타임스텝의 출력만 사용
        return out

# 모델, 손실 함수, 옵티마이저 초기화
model = SimpleRNN(input_size, hidden_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 임의의 입력 데이터와 레이블
inputs = torch.randn(batch_size, sequence_length, input_size)
labels = torch.tensor([0, 1, 0]) # 3개의 레이블 : BATCHSIZE = 3

# 학습
for epoch in range(num_epochs):
    outputs = model(inputs)
    loss = criterion(outputs, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 5 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [5/20], Loss: 0.4280
Epoch [10/20], Loss: 0.2265
Epoch [15/20], Loss: 0.0801
Epoch [20/20], Loss: 0.0264


### multi layer RNN

In [ ]:
num_layers = 4
rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
outputs, hidden = rnn(inputs) # inputs: (batch_size, seq_length, input_size)
print("Output shape:", outputs.shape) # (batch_size, seq_length, hidden_size)
print("Hidden shape:", hidden.shape) # (num_layers, batch_size, hidden_size)

Output shape: torch.Size([3, 6, 10])
Hidden shape: torch.Size([4, 3, 10])


In [ ]:
class MultiLayerRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(MultiLayerRNN, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :]) #batch, seq, hidden
        return out

num_layers = 4
model = MultiLayerRNN(input_size, hidden_size, num_layers, num_classes)

# 모델, 손실 함수, 옵티마이저 초기화
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 임의의 입력 데이터와 레이블
inputs = torch.randn(batch_size, sequence_length, input_size)
labels = torch.tensor([0, 1, 0])

# 학습
for epoch in range(num_epochs):
    outputs = model(inputs)
    loss = criterion(outputs, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 5 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [5/20], Loss: 0.5117
Epoch [10/20], Loss: 0.1344
Epoch [15/20], Loss: 0.0332
Epoch [20/20], Loss: 0.0134


### 양방향 RNN

In [ ]:
rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
outputs, hidden = rnn(inputs)

# input_size = 5
# hidden_size = 10
# sequence_length = 6
# batch_size = 3
# num_layers = 4
print("Output shape:", outputs.shape) # (batch_size, seq_length, hidden_size * 2)
print("Hidden shape:", hidden.shape) # (num_layers * 2, batch_size, hidden_size)

Output shape: torch.Size([3, 6, 20])
Hidden shape: torch.Size([8, 3, 10])


In [ ]:
# RNN의 hidden은 첫번째 레이어부터 마지막 레이어까지 forward, backward가 순차적으로 쌓인 것
out_forward = outputs[:, -1, :hidden_size]  # forward 방향의 마지막 타임스텝 출력
out_backward = outputs[:, 0, hidden_size:]  # backward 방향의 첫 번째 타임스텝 출력 (뒤에서 앞으로)

# forward와 backward 방향의 출력을 결합
out_combined = torch.cat((out_forward, out_backward), dim=1)

In [ ]:
# 또는 hidden을 이용할 수도 있음
#https://stackoverflow.com/questions/63121983/bidirectional-rnn-implementation-pytorch
out_combined = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)

In [ ]:
class BiDirectionalRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(BiDirectionalRNN, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, num_classes)  # 양방향이므로 Linear에 넣어야 하는 히든 크기가 2배

    def forward(self, x):
        out, hidden = self.rnn(x)

        # hidden[-2,:,:]: forward의 마지막 레이어 마지막 타임스텝 히든 상태
        # hidden[-1,:,:]: backward의 마지막 레이어 마지막 타임스텝 히든 상태
        out_combined = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)  # 마지막 타임스텝의 출력만 사용
        out = self.fc(out_combined)
        return out

# 모델 초기화 (양방향 설정)
model = BiDirectionalRNN(input_size, hidden_size, num_layers=2, num_classes=num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 임의의 입력 데이터와 레이블
inputs = torch.randn(batch_size, sequence_length, input_size)
labels = torch.tensor([0, 1, 0])

# 학습
for epoch in range(num_epochs):
    outputs = model(inputs)
    loss = criterion(outputs, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 5 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [5/20], Loss: 0.3456
Epoch [10/20], Loss: 0.0788
Epoch [15/20], Loss: 0.0184
Epoch [20/20], Loss: 0.0057
